## Produce weighted plots showing data, signal and background processes

Running CutLang provides ```histoOut-<adlname>.root``` files which contain histograms available for each analysis region.
This notebook provides code based on ```PyROOT``` to make plots comparing distributions between two files.

The code will plot data, signal and background histograms after applying event weights that will normalize them to cross section times luminosity.  The background processes are drawn in a stacked histogram (processes are added on top of each other).

The code requires the following input
  * A directory containing a set of ```histoOut-<adlname>_<samplename>.root``` files and a ```samples.txt``` file that lists the each sample in a line which includes sample process category (e.g. Top, QCD, Data, signal, ...), ```histoOut-<adlname>_<samplename>.root``` file name, cross section and unskimmed number of events (corresponding to the quoted cross section).
  * A python dictionary listing sample process category and ROOT colors for plotting (see cell 2).
  * Python lists specifying which sample process categories are signals and which ones are backgrounds (see cell 2).
  * A list of regions and histograms you would like to draw.

In [ ]:
# Let's start with importing the needed modules
from ROOT import gStyle, TFile, TH1, TH1D, TH2D, THStack, TCanvas, TLegend, TColor, TLatex, SetOwnership
import random
from string import *
# Now let's set some ROOT styling parameters:
# You do not need to know what they mean, but can directly use these settings

gStyle.SetOptStat(0)
gStyle.SetPalette(1)

gStyle.SetTextFont(42)

gStyle.SetTitleStyle(0000)
gStyle.SetTitleBorderSize(0)
gStyle.SetTitleFont(42)
gStyle.SetTitleFontSize(0.055)

gStyle.SetTitleFont(42, "xyz")
gStyle.SetTitleSize(0.5, "xyz")
gStyle.SetLabelFont(42, "xyz")
gStyle.SetLabelSize(0.45, "xyz")

# A function to apply a fix to the cutflow histograms from CutLang.  
# It takes out the redundant bins showing histogramming.
def fixcutflow(h):
    nbins = 0
    for i in range(1, h.GetXaxis().GetNbins()+1):
        if not ("Histo" in h.GetXaxis().GetBinLabel(i) or "ALL" in h.GetXaxis().GetBinLabel(i)):
            nbins = nbins + 1
    hnew = TH1D(h.GetName()+str(random.random()), h.GetTitle(), nbins, 0, nbins)
    ibin = 0
    for i in range(1, h.GetXaxis().GetNbins()+1):
        label = h.GetXaxis().GetBinLabel(i)
        if not ("Histo" in label or "ALL" in label):
            ibin = ibin + 1
            hnew.GetXaxis().SetBinLabel(ibin, label)
            hnew.SetBinContent(ibin, h.GetBinContent(i))
    h = hnew
    SetOwnership(h, False)
    return h


In [ ]:
# Define processes to use, colors and lumi

# Location of the directory containing histoOut-<adlname>.root files and samples.txt:
samplesdir = '../../src/ttbartovlq-results/'

# Put the samples in a list:
fin = open(samplesdir+'samples.txt').readlines()

# Dictionary listing sample process categories and ROOT histogram colors
samples = {
    'Data' : 1, # kBlack
    'TT1200' : 400+2, # kYellow+2
    'TT800' : 600+1, # kBlue+1
    'TT700' : 432+2, # kCyan+2
    'Top' : 860+6, # kAzure+6
    'QCD' : 632, # kRed
    'EWK' : 616+1, # kMagenta+1
}

# List of background process categories (the first one is drawn lowest in the stack)
bgs = ['QCD', 'EWK', 'Top']
# List of signal process categories
sgs = ['TT700', 'TT800', 'TT1200']
# Luminosity
lumi = 2.26

In [ ]:
# Read the samples file and histoOut files and add relevant information to the dictionary:
# Each process gets assigned a list of samples belonging to it.  
# In the samples list, each line has the information process name, histoOut file, xsec, 
#   number of events corresponding to the xsec, and event weight. 
for key in samples:
    col = samples[key]
    smplist = []
    for l in fin:
        if "#" in l or len(l) < 2: continue
        l = l.strip().split()
        if l[0] == key:
            l[1] = l[1]+'.root'
            l.append(TFile(samplesdir+l[1]))
            xsec = float(l[2])
            nevt = float(l[3])
            wgt = lumi * xsec*1000  / nevt
            if xsec == -1: wgt = 1
            l.append(wgt)
            smplist.append(l)
            samples[key] = col, smplist

In [ ]:
# Now let's draw some histograms. 
# We will compare distributions for different variables.
# You can try this with different histograms in different regions.
# Provide the region and histo names in the following list:
histoinfos = [
# ["<regionname>", "<histoname>"]
    ["fourjetoneb", "cutflow"],
    ["fourjetoneb", "hmtop1b"],
    ["fourjetoneb", "h2mtop1b"],
    ["fourjettwob", "cutflow"],
    ["fourjettwob", "hmtop"],
    ["fourjettwob", "h2mtop"],
    ["fourjettwob", "hpttop"],
    ["fourjettwob", "h2pttop"],
    ["fourjettwob", "hnjets"],
    ["fourjettwob", "hj1pT"],
    ["fourjettwob", "hj2pT"],
    ["fourjettwob", "hj3pT"],
    ["fourjettwob", "hmet"],
    ["fourjettwob", "hST"],
    ["fourjettwob", "hnak8"],
    ["optforvlqreint", "cutflow"],
    ["optforvlqreint", "hak8j1pT"],
    ["optforvlqreint", "h2ak8j1pT"],
    ["optforvlqreint", "hak8j1m"],
    ["optforvlqreint", "h2ak8j1m"],
    ["optforvlqreint", "h2mtop2"],
    ["optforvlqreint", "h2pttop2"],
    ["optforvlqreint", "hmet2"],
    ["optforvlqreint", "hST2"],
    ["optforvlqreint", "h2ak8j1pT2"],
    ["optforvlqreint", "h2ak8j1m2"],
]


In [ ]:
# Make lists for convases and legends. This is just a fix for jupyter root!!!
canvases = []
texts = []
legends = []

# Loop over the histograms and prepare the plots:
for hinfo in histoinfos:
    # In which region would you like to draw? You can change the region name. 
    region = hinfo[0]
    # Which histogram would you like to plot?
    hname = hinfo[1]
    print(region+"/"+hname)
    
    # Make a plot legend
    # Change the entry names to reflect processes you are plotting!
    leg = TLegend(0.52, 0.70, 0.85, 0.82)
    leg.SetBorderSize(0)
    leg.SetFillStyle(0000)
    leg.SetNColumns(2)
    
    # Manage the data histograms
    col, smplist = samples['Data']
    hdt = smplist[1][4].Get(region+"/"+hname)
    hdt.SetName(hdt.GetName()+str(random.random()))
    for i in range(len(smplist)):
        if i == 0: continue
        hdt.Add(smplist[i][4].Get(region+"/"+hname))
    if hname == "cutflow": hdt = fixcutflow(hdt)
    hdt.SetMarkerColor(col)
    hdt.SetMarkerStyle(20)
    leg.AddEntry(hdt, "Data", "p")

    # Manage the background histograms.  These are put in a stacked histogram.
    hbg = THStack("hbg"+str(random.random()),"hbg"+str(random.random()))
    for bg in bgs:
        col, smplist = samples[bg]
        for l in smplist:
            hbgi = l[4].Get(region+"/"+hname)
            hbgi.SetName(hbgi.GetName()+str(random.random()))
            hbgi.Scale(l[5])
            if hname == "cutflow": hbgi = fixcutflow(hbgi)
            hbgi.SetLineColor(col)
            hbgi.SetFillColor(col)
            SetOwnership(hbg, False)            
            hbg.Add(hbgi)
        leg.AddEntry(hbgi, bg, "f")
        
    # Manage the signal histograms
    hsgs = []
    for sg in sgs:
        col, smplist = samples[sg]
        hsg = smplist[0][4].Get(region+"/"+hname)
        hsg.SetName(hsg.GetName()+str(random.random()))
        hsg.Scale(smplist[0][5])
        if hname == "cutflow": hsg = fixcutflow(hsg)
        hsg.SetLineColor(col)
        hsg.SetLineWidth(3)
        leg.AddEntry(hsg, sg, "l")
        hsgs.append(hsg)

    # Legend is complete.  Add it to the legends list
    legends.append(leg)
        
    # Find the maximum
    mxm = max(hbg.GetMaximum(), hdt.GetMaximum())*1.2

    # Style for the first histogram to be drawn
    hsgs[0].SetTitle("")
    hsgs[0].SetMaximum(mxm)
    hsgs[0].GetXaxis().SetTitle(hdt.GetTitle())
    hsgs[0].GetXaxis().SetTitleOffset(1.25)
    hsgs[0].GetXaxis().SetTitleSize(0.05)
    hsgs[0].GetXaxis().SetLabelSize(0.045)
    hsgs[0].GetXaxis().SetNdivisions(8, 5, 0)
    hsgs[0].GetYaxis().SetTitle("Number of events")
    hsgs[0].GetYaxis().SetTitleOffset(1.4)
    hsgs[0].GetYaxis().SetTitleSize(0.05)
    hsgs[0].GetYaxis().SetLabelSize(0.045)
        
    # Write the region name on the histogram
    t = TLatex(0.60, 0.85, region)
    t.SetTextSize(0.041)
    t.SetTextFont(42)
    t.SetNDC()
    texts.append(t)

    # Now we make a canvas and draw our histograms
    cxwidth = 650
    if hname == "cutflow": cxwidth = 1000
    c = TCanvas("c_"+region+"_"+hname+str(random.random()), "c_"+region+"_"+hname, cxwidth, 500)
    c.SetBottomMargin(0.15)
    c.SetLeftMargin(0.15)
    c.SetRightMargin(0.15)
    logy = 1
    if logy == 1:
        hsgs[0].SetMaximum(hsgs[0].GetMaximum()*3)
        hsgs[0].SetMinimum(0.5)
    c.SetLogy(logy)
    hsgs[0].Draw("hist")
    hbg.Draw("hist same")
    hdt.Draw("e same")
    for hsg in hsgs:
        hsg.Draw("hist same")
        hsg.Draw("axis same")    
    leg.Draw("same")
    t.Draw("same")
    canvases.append(c)
        
# Draw the canvases:
for c in canvases:
    c.Draw()
